In [1]:
import json
import numpy as np
from tqdm import tqdm
import peewee
import time
import datetime
import pandas as pd

In [2]:
from src import orm

In [3]:
with open('data/bgg_data.json', 'r') as f:
    games = json.load(f)
games[0]

{'name': 'CATAN',
 'release_year': 1995,
 'category': ['Economic', 'Negotiation'],
 'mechanic': ['Dice Rolling',
  'Hexagon Grid',
  'Income',
  'Modular Board',
  'Negotiation',
  'Network and Route Building',
  'Race',
  'Random Production',
  'Trading',
  'Variable Set-up'],
 'players_min': 3,
 'players_max': 4,
 'playtime_min': 60,
 'playtime_max': 120,
 'min_age': 10,
 'is_expansion': False}

In [4]:
','.join(games[0]['category'])

'Economic,Negotiation'

In [5]:
# Insert data to database
for game in games:
    game["category"] = ','.join(game["category"])
    game["mechanic"] = ','.join(game["mechanic"])
orm.Games.insert_many(games).execute()

## Generate Games To Sell

In [6]:
# Create new GameToSell for each game in Game
games_to_sell = []
for game in tqdm(orm.Games.select()):
    games_to_sell.append({
        "game_id": game.id,
        "price": round(max(min(np.random.normal(100, 30), 200.0), 60.0), 2),
        "amount": np.random.randint(0, 5)
    })
orm.GamesToSell.insert_many(games_to_sell).execute()


100%|██████████| 234/234 [00:00<00:00, 116994.53it/s]


## Generate employees

In [7]:
first_day_timestamp = '2018-06-01'

In [8]:
orm.Employees.insert_many([
    {
        'first_name': 'Marzena', 'last_name': 'Rybak', 'email': '268951@student.pwr.edu.pl', 'phone': '538692385',
        'hired_date': first_day_timestamp, 'position': 'manager', 'date_end_of_work': None, 'salary': None
    },
    {
        'first_name': 'Adrian', 'last_name': 'Radziewicz', 'email': '268816@student.pwr.edu.pl', 'phone': '555333111',
        'hired_date': first_day_timestamp, 'position': 'mangager', 'date_end_of_work': None, 'salary': 4000.0
    },
    {
        'first_name': 'Damian', 'last_name': 'Bojarun', 'email': '268814@student.pwr.edu.pl', 'phone': '213721379',
        'hired_date': first_day_timestamp, 'position': 'mangager', 'date_end_of_work': None, 'salary': 4500.0
    },
    {
        'first_name': 'Jakub', 'last_name': 'Borcoń', 'email': '268834@student.pwr.edu.pl', 'phone': '556478789',
        'hired_date': first_day_timestamp, 'position': 'mangager', 'date_end_of_work': None, 'salary': 4499.0
    }
]).execute()

In [9]:
top_first_names = pd.read_csv("./data/new-top-firstNames.csv", sep=",")
top_last_names = pd.read_csv("./data/new-top-surnames.csv", sep=",")
top_first_names["newPerct2013"] = top_first_names["newPerct2013"] / top_first_names["newPerct2013"].sum()
top_last_names["newPerct2013"] = top_last_names["perct2013"] / top_last_names["perct2013"].sum()

def get_random_name():
    first_name = np.random.choice(top_first_names["name"], p=top_first_names["newPerct2013"])
    last_name = np.random.choice(top_last_names["name"], p=top_last_names["newPerct2013"])
    # last name is uppercased - fix that
    last_name = last_name[0] + last_name[1:].lower()
    return first_name, last_name
get_random_name()

('Mary', 'Parker')

In [10]:
cashiers = []
phone_numbers = set(orm.Employees.select(orm.Employees.phone))
stopped_working = 0

for _ in range(5):
    first_name, last_name = get_random_name()
    phone = "".join([str(np.random.randint(0, 10)) for _ in range(9)])
    if phone[0] == "0":
        phone = str(np.random.randint(1, 10)) + phone[1:]
    while phone in phone_numbers:
        phone = "".join([str(np.random.randint(0, 10)) for _ in range(9)])
        if phone[0] == "0":
            phone = str(np.random.randint(1, 10)) + phone[1:]
    phone_numbers.add(phone)

    email_variant = np.random.choice(["first,last", "first,dot,last", "first,numbers", 
                                      "last,first", "last,numbers", "first,last,numbers", 
                                      "last,first,numbers", "last,dot,first", "first,dot,last,numbers", "last,dot,first,numbers"
                                      ])

    email = ""
    for email_part in email_variant.split(","):
        if email_part == "first":
            email += first_name
        elif email_part == "last":
            email += last_name
        elif email_part == "numbers":
            email += "".join([str(np.random.randint(0, 10)) for _ in range(np.random.randint(1, 4))])
        elif email_part == "dot":
            email += "."

    # Add email provider from gmail outlook etc
    email += "@" + np.random.choice(["gmail", "outlook", "yahoo", "protonmail", "tutanota"]) + ".com"

    # Get random date between 2018-06-01 and 2019-06-01
    date_start_work = datetime.datetime(2018, 7, 20) + datetime.timedelta(days=np.random.randint(0, 365))
    date_end_work = None
    if np.random.rand() < 0.3 and stopped_working < 2:
        stopped_working += 1
        date_end_work = date_start_work + datetime.timedelta(days=np.random.randint(365, 765))

    cashiers.append({
        'first_name': first_name, 'last_name': last_name, 'email': email, 'phone': phone,
        'hired_date': date_start_work, 
        'position': 'cashier', 
        'date_end_of_work': date_end_work, 
        'salary': np.random.choice([3600, 3650, 3700, 3750, 3800])
    })


In [11]:
orm.Employees.insert_many(cashiers).execute()

## Generate sales history

In [12]:
games_to_sell_ids = [game.id for game in orm.GamesToSell.select()]
employees = [employee for employee in orm.Employees.select()]

In [13]:
shop_start_date = datetime.date(2018, 6, 1)
covid_start_date = datetime.date(2020, 3, 12)
covid_end_date = datetime.date(2020, 5, 4)
shop_end_date = datetime.date(2023, 6, 28)

In [14]:
employees[0].hired_date.strftime("%Y-%m-%d")

'2018-06-01'

In [15]:
sales = []
for day in tqdm(range((covid_start_date - shop_start_date).days)):
    # If this is a Sunday skip
    if (shop_start_date + datetime.timedelta(days=day)).weekday() == 6:
        continue
    this_date_str = (shop_start_date + datetime.timedelta(days=day)).strftime("%Y-%m-%d")
    # Filter out employees that were not hired this day
    employees_ids = []
    for employee in employees:
        if employee.hired_date.strftime("%Y-%m-%d") <= this_date_str and (employee.date_end_of_work is None or employee.date_end_of_work.strftime("%Y-%m-%d") > this_date_str):
            employees_ids.append(employee.id)
    employees_this_day = np.random.choice(employees_ids, size=4, replace=False)
    no_sales = np.random.randint(10, 30)
    for _ in range(no_sales):
        sales.append({
            "employee_id": np.random.choice(employees_this_day),
            "game_id": np.random.choice(games_to_sell_ids),
            "date": this_date_str + " " + str(np.random.randint(10, 18)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60)),
        })

for day in tqdm(range((shop_end_date - covid_end_date).days)):
    # If this is a Sunday skip
    if (covid_end_date + datetime.timedelta(days=day)).weekday() == 6:
        continue
    this_date_str = (covid_end_date + datetime.timedelta(days=day)).strftime("%Y-%m-%d")
    # Filter out employees that were not hired this day
    employees_ids = []
    for employee in employees:
        if employee.hired_date.strftime("%Y-%m-%d") <= this_date_str and (employee.date_end_of_work is None or employee.date_end_of_work.strftime("%Y-%m-%d") > this_date_str):
            employees_ids.append(employee.id)
    employees_this_day = np.random.choice(employees_ids, size=4, replace=False)
    no_sales = np.random.randint(10, 30)
    for _ in range(no_sales):
        sales.append({
            "employee_id": np.random.choice(employees_this_day),
            "game_id": np.random.choice(games_to_sell_ids),
            "date": this_date_str + " " + str(np.random.randint(10, 18)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60)),
        })

100%|██████████| 1150/1150 [00:01<00:00, 948.42it/s]


In [16]:
# Sort sales by date
sales = sorted(sales, key=lambda x: x["date"])

In [17]:
# Insert sales into db
orm.Sales.insert_many(sales).execute()

## Generate additional payment

In [18]:
orm.AdditionalPayment.insert_many([
    {'days_paid_time_min': 0,  'days_paid_time_max': 14, 'payment_percent': 0.0},
    {'days_paid_time_min': 14, 'days_paid_time_max': 21, 'payment_percent': 0.1},
    {'days_paid_time_min': 21, 'days_paid_time_max': 28, 'payment_percent': 0.2},
    {'days_paid_time_min': 28, 'days_paid_time_max': 999, 'payment_percent': 0.5},
]).execute()

## Generate GamesToRent

In [19]:
# Generate GamesToRent based on Games
games_to_rent = []
for game in orm.Games.select():
    # Find corresponding game in GamesToSell
    game_to_sell = orm.GamesToSell.get(orm.GamesToSell.game_id == game.id)

    for _ in range(np.random.randint(1, 10)):
        is_discounted = np.random.choice([True, False], p=[0.1, 0.9])
        date = (shop_start_date + datetime.timedelta(days=np.random.randint(0, 365))).strftime("%Y-%m-%d")
        games_to_rent.append({
            "game_id": game.id,
            "price": round(game_to_sell.price * 0.1, 2),
            "discount": 0.3 if is_discounted else 0,
            "date": date + " " + str(np.random.randint(10, 17)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60)),
            "comments": "Missing parts" if is_discounted else "",
            "rented": False
        })

In [20]:
games_to_rent[2]

{'game_id': 1,
 'price': 10.64,
 'discount': 0,
 'date': '2019-01-13 13:25:17',
 'comments': '',
 'rented': False}

In [21]:
games_to_rent = sorted(games_to_rent, key=lambda x: x["date"])

In [22]:
# Insert GamesToRent into db
orm.GamesToRent.insert_many(games_to_rent).execute()

## Generate customers

In [23]:
top_first_names = pd.read_csv("./data/new-top-firstNames.csv", sep=",")
top_last_names = pd.read_csv("./data/new-top-surnames.csv", sep=",")

In [24]:
top_first_names.head()

,Unnamed: 0,name,newPerct2013
0,1,Michael,0.011577
1,2,James,0.010218
2,3,John,0.009675
3,4,Robert,0.009493
4,5,David,0.008943


In [25]:
top_first_names["newPerct2013"] = top_first_names["newPerct2013"] / top_first_names["newPerct2013"].sum()
top_last_names["newPerct2013"] = top_last_names["perct2013"] / top_last_names["perct2013"].sum()

In [26]:
def get_random_name():
    first_name = np.random.choice(top_first_names["name"], p=top_first_names["newPerct2013"])
    last_name = np.random.choice(top_last_names["name"], p=top_last_names["newPerct2013"])
    # last name is uppercased - fix that
    last_name = last_name[0] + last_name[1:].lower()
    return first_name, last_name
get_random_name()

('Mary', 'Gray')

In [27]:
customers = []
phone_numbers = set(orm.Employees.select(orm.Employees.phone))

no_customers = 1000
for _ in tqdm(range(no_customers)):
    first_name, last_name = get_random_name()

    phone = "".join([str(np.random.randint(0, 10)) for _ in range(9)])
    if phone[0] == "0":
        phone = str(np.random.randint(1, 10)) + phone[1:]
    while phone in phone_numbers:
        phone = "".join([str(np.random.randint(0, 10)) for _ in range(9)])
        if phone[0] == "0":
            phone = str(np.random.randint(1, 10)) + phone[1:]
    phone_numbers.add(phone)

    email_variant = np.random.choice(["first,last", "first,dot,last", "first,numbers", 
                                      "last,first", "last,numbers", "first,last,numbers", 
                                      "last,first,numbers", "last,dot,first", "first,dot,last,numbers", "last,dot,first,numbers"
                                      ])

    email = ""
    for email_part in email_variant.split(","):
        if email_part == "first":
            email += first_name
        elif email_part == "last":
            email += last_name
        elif email_part == "numbers":
            email += "".join([str(np.random.randint(0, 10)) for _ in range(np.random.randint(1, 4))])
        elif email_part == "dot":
            email += "."

    # Add email provider from gmail outlook etc
    email += "@" + np.random.choice(["gmail", "outlook", "yahoo", "protonmail", "tutanota"]) + ".com"

    customers.append({
        "first_name": first_name,
        "last_name": last_name,
        "phone": phone,
        "email": email,
        "discount": 0.0,
        # Make this temporary date between shop start and end date
        "date_of_registration": shop_start_date + datetime.timedelta(days=np.random.randint(0, (shop_end_date - shop_start_date).days)),
    })

100%|██████████| 1000/1000 [00:00<00:00, 4300.65it/s]


In [28]:
# Sort customers
customers = sorted(customers, key=lambda x: x["date_of_registration"])

In [29]:
covid_len = (covid_end_date - covid_start_date)
# Iterate over customers and fix dates that are in covid
for customer in customers:
    first_rental = customer["date_of_registration"]
    # Check if date is in covid
    if first_rental >= covid_start_date and first_rental <= covid_end_date:
        # Fix date
        # If its before middle of covid move it pre covid
        if first_rental < covid_start_date + covid_len / 2:
            customer["date_of_registration"] = covid_start_date - datetime.timedelta(days=np.random.randint(0, 64))
        elif first_rental >= covid_start_date + covid_len / 2:
            customer["date_of_registration"] = covid_end_date + datetime.timedelta(days=np.random.randint(0, 64))
    # Get str from date
    customer["date_of_registration"] = customer["date_of_registration"].strftime("%Y-%m-%d")
    # Add random hours and minutes and seconds to date
    new_date =  customer["date_of_registration"] + " " + str(np.random.randint(10, 17)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60))

    customer["date_of_registration"] = new_date

In [30]:
customers = sorted(customers, key=lambda x: x["date_of_registration"])
customers[0]

{'first_name': 'Donald',
 'last_name': 'Price',
 'phone': '563589581',
 'email': 'DonaldPrice184@outlook.com',
 'discount': 0.0,
 'date_of_registration': '2018-06-09 10:4:4'}

In [31]:
# Insert customers into db
orm.Customers.insert_many(customers).execute()

## Prizes

In [32]:
# This should be generated with tournaments

## Simulate Rental

In [33]:
import simpy as sp
from copy import copy, deepcopy

In [34]:
sim_data = []
env = sp.Environment()
customers = orm.Customers.select()
games_to_rent = orm.GamesToRent.select()

In [35]:
len(customers), len(games_to_rent)

(1000, 1162)

In [36]:
class SimGameToRent(object):
    def __init__(self, env, game_data):
        self.resource = sp.Resource(env, capacity=1)
        self.count = 0
        self.game_data = game_data

    def after_return(self, current_day):
        self.count += 1
        if self.count > 10:
            if np.random.random() < 0.1:
                self.game_data.date_of_delete = current_day
        if self.count > 25:
            self.game_data.date_of_delete = current_day

In [37]:
class GameListContainer():
    def __init__(self, games_to_rent) -> None:
        self.game_list = games_to_rent
        self.removed_games = []
        self._games = orm.Games.select()

    def remove_game(self, game):
        self.game_list.remove(game)
        self.removed_games.append(game)

    def add_game(self, curr_date):
        # Get random game from orm.Games
        game = np.random.choice(self._games)
        # Get price from orm.GamesToSell
        new_price = orm.GamesToSell.select().where(orm.GamesToSell.game_id == game.id).get().price
        new_date = curr_date + datetime.timedelta(days=np.random.randint(1, 4))
        # Check if it is sunday
        if new_date.weekday() == 6:
            new_date += datetime.timedelta(days=1)
        new_game = {
            "game_id": game.id,
            # Add this game in 1-3 days
            "date": new_date,
            "date_of_delete": None,
            "price": new_price * 0.1,
            "discount": 0.0,
            "comments": "",
            "rented": False,
        }
        new_game = orm.GamesToRent(**new_game)

        new_game = SimGameToRent(env, new_game)

        print("Adding new game: ", game.name, " ", new_date)
        self.game_list.append(new_game)
        return new_game

    def get_random_game(self, curr_date):
        res = None
        retries = 5
        while res is None and retries > 0:
            tmp = np.random.choice(self.game_list)
            retries -= 1
            # Check if game date is not in future
            if tmp.game_data.date > curr_date:
                continue
            if tmp.game_data.date_of_delete is None and tmp.resource.count == 0:
                res = tmp
            else:
                # Get base game from orm.Games
                game = orm.Games.select().where(orm.Games.id == tmp.game_data.game_id).get()
                print("Removing game: ", game.name, " ", curr_date)
                self.remove_game(tmp)
                tmp.game_data.date_of_delete = curr_date
                res = self.add_game(curr_date)
        return res

In [38]:
# Create SimGameToRent objects
sim_games_to_rent = []
for game in games_to_rent:
    sim_games_to_rent.append(SimGameToRent(env, game))

In [39]:
game_list_container = GameListContainer(sim_games_to_rent)

In [40]:
class SimCustomer(object):
    def __init__(self, env, customer_data, games_container):
        self.env = env
        self.customer_data = customer_data
        self.rented_game = None
        self.rented_games_history = []

        self.games_container = games_container

        self.action = env.process(self.run())

    def run(self):
        while True:
            first_rental_date = self.customer_data.date_of_registration
            current_time = shop_start_date + datetime.timedelta(days=int(self.env.now))
            # Timeout until first rental if current time is before first rental
            if current_time < first_rental_date:
                yield self.env.timeout((first_rental_date - shop_start_date).days)
                continue

            # Check if current time is in covid
            if current_time >= covid_start_date and current_time <= covid_end_date:
                yield self.env.timeout(1)
                continue

            # Check if current day is sunday
            if current_time.weekday() == 6:
                yield self.env.timeout(1)
                continue

            game = self.games_container.get_random_game(current_time)
            if game is None:
                yield self.env.timeout(1)
                continue
            with game.resource.request():
                current_time = shop_start_date + datetime.timedelta(days=int(self.env.now))
                # Copy current time
                rent_time = deepcopy(current_time)
                # Borrow game for random time
                return_week = np.random.choice([0, 1, 2, 3, 4], p=[0.9, 0.05, 0.02, 0.02, 0.01])
                yield self.env.timeout(np.random.randint(1, 8) + return_week * 7)

                # Recalculate current time
                current_time = shop_start_date + datetime.timedelta(days=int(self.env.now))
                # if this is sunday, add or remove 1 day
                if current_time.weekday() == 6:
                    current_time += datetime.timedelta(days=1 if np.random.random() < 0.5 else -1)
                self.rented_games_history.append((game.game_data, rent_time, deepcopy(current_time)))
                # Return game
                game.after_return(current_time)

            if np.random.random() < 0.5:
                # With 30% chance wait max 2 weeks
                if np.random.random() < 0.3:
                    yield self.env.timeout(np.random.randint(2, 14))
                    continue
                yield self.env.timeout(np.random.randint(14, 30))
            else:
                # In 40% of left cases never come back
                if np.random.random() < 0.4:
                    yield self.env.timeout(10000)
                    break


In [41]:
# Create customers list
sim_customers = []
for customer in customers:
    sim_customers.append(SimCustomer(env, customer, game_list_container))

In [42]:
# Cast shop dates to datetime
shop_start_date = datetime.datetime(shop_start_date.year, shop_start_date.month, shop_start_date.day)
covid_start_date = datetime.datetime(covid_start_date.year, covid_start_date.month, covid_start_date.day)
covid_end_date = datetime.datetime(covid_end_date.year, covid_end_date.month, covid_end_date.day)
shop_end_date = datetime.datetime(shop_end_date.year, shop_end_date.month, shop_end_date.day)
shop_start_date

datetime.datetime(2018, 6, 1, 0, 0)

In [43]:
for t in tqdm(range(1, (shop_end_date - shop_start_date).days)):
    env.run(until=t)

 10%|█         | 193/1852 [00:00<00:05, 311.76it/s]

Removing game:  Sushi Go Party!   2018-11-28 00:00:00
Adding new game:  Mascarade (second edition)   2018-12-01 00:00:00


 19%|█▉        | 353/1852 [00:01<00:04, 365.15it/s]

Removing game:  Dune   2019-03-12 00:00:00
Adding new game:  Cloudspire   2019-03-15 00:00:00
Removing game:  My Father's Work   2019-04-19 00:00:00
Adding new game:  Nova Luna   2019-04-22 00:00:00
Removing game:  Woodcraft   2019-04-20 00:00:00
Adding new game:  The Witcher: The Adventure Card Game   2019-04-23 00:00:00


 28%|██▊       | 522/1852 [00:01<00:02, 501.27it/s]

...

 92%|█████████▏| 1695/1852 [00:03<00:00, 486.80it/s]

Removing game:  The Resistance: Avalon   2022-11-07 00:00:00
Adding new game:  Fantasy Realms   2022-11-10 00:00:00
Removing game:  Whitehall Mystery   2022-11-17 00:00:00
Adding new game:  Palm Island   2022-11-19 00:00:00
Removing game:  Dixit   2022-12-23 00:00:00
Adding new game:  War Chest   2022-12-24 00:00:00


 99%|█████████▉| 1842/1852 [00:03<00:00, 418.04it/s]

Removing game:  Kingdom Builder   2023-04-03 00:00:00
Adding new game:  Ticket to Ride: Rails & Sails   2023-04-05 00:00:00
Removing game:  Ticket to Ride: Rails & Sails   2023-04-06 00:00:00
Adding new game:  Underwater Cities   2023-04-07 00:00:00
Removing game:  Skull King   2023-05-09 00:00:00
Adding new game:  Roll Player   2023-05-12 00:00:00


100%|██████████| 1852/1852 [00:03<00:00, 471.70it/s]


In [44]:
sim_customers[1].rented_games_history

[(<GamesToRent: 19>,
  datetime.datetime(2018, 6, 18, 0, 0),
  datetime.datetime(2018, 6, 25, 0, 0)),
 (<GamesToRent: 76>,
  datetime.datetime(2018, 6, 26, 0, 0),
  datetime.datetime(2018, 6, 30, 0, 0)),
 (<GamesToRent: 105>,
  datetime.datetime(2018, 7, 9, 0, 0),
  datetime.datetime(2018, 7, 13, 0, 0)),
 (<GamesToRent: 42>,
  datetime.datetime(2018, 7, 31, 0, 0),
  datetime.datetime(2018, 8, 6, 0, 0))]

In [45]:
game_list_container.removed_games[0].game_data.date_of_delete

datetime.datetime(2018, 11, 28, 0, 0)

In [46]:
# Count total of new games (without id)
new_games = 0
for game in game_list_container.game_list:
    if game.game_data.id is None:
        new_games += 1
print(new_games)
print(len(game_list_container.removed_games))

20
21


In [47]:
# Print number of rentals
rentals = 0
for customer in sim_customers:
    rentals += len(customer.rented_games_history)
print(rentals)

2594


In [48]:
# Update database
for game in game_list_container.removed_games:
    game.game_data.save()

for game in game_list_container.game_list:
    game.game_data.save()

In [49]:
additional_payments = orm.AdditionalPayment.select()

In [50]:
additional_payments[0].payment_percent + 0.7

0.7

In [51]:
def get_correct_additional_payment(days):
    for additional_payment in additional_payments:
        if days >= additional_payment.days_paid_time_min and days < additional_payment.days_paid_time_max:
            return additional_payment

In [52]:
rental_history = []
for customer in sim_customers:
    for game, rent_date, return_date in customer.rented_games_history:
        rented_for_days = int((return_date - rent_date).days)
        if np.random.random() < 0.1:
            ap = get_correct_additional_payment(1)
        else:
            ap = get_correct_additional_payment(rented_for_days)

        rent_date = rent_date.strftime("%Y-%m-%d") + " " + str(np.random.randint(10, 17)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60))
        return_date = return_date.strftime("%Y-%m-%d") + " " + str(np.random.randint(10, 17)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60))
        
        
        
        rental_history.append({
            "customer_id": customer.customer_data.id,
            "game_id": game.id,
            "rental_date": rent_date,
            "return_date": return_date,
            "additional_payment_id": ap.id,
            # "employee_id"
        })

In [53]:
# Sort rental by rental_date
rental_history = sorted(rental_history, key=lambda x: x["rental_date"])

In [54]:
# Get sales data
sales = orm.Sales.select()

In [55]:
sales[0].date

datetime.datetime(2018, 6, 1, 10, 12, 53)

In [56]:
employees_per_day = {}
for sale in sales:
    date = sale.date.strftime("%Y-%m-%d")
    if date not in employees_per_day:
        employees_per_day[date] = set()
    employees_per_day[date].add(sale.employee_id)

# Recast each set to list
for key in employees_per_day:
    employees_per_day[key] = list(employees_per_day[key])

In [57]:
employees = orm.Employees.select()

In [58]:
# Go for each day from shop start date to shop end date
current_date = shop_start_date
while current_date <= shop_end_date:
    date_as_str = current_date.strftime("%Y-%m-%d")
    if date_as_str not in employees_per_day:
        employees_per_day[date_as_str] = []
    # Check if there are 2 employees on current day
    if len(employees_per_day[date_as_str]) == 4:
        current_date += datetime.timedelta(days=1)
        continue
    # Add random emploees so it will be 4
    while len(employees_per_day[date_as_str]) < 4:
        random_employee = np.random.choice(employees)
        if random_employee.id not in employees_per_day[date_as_str]:
            employees_per_day[date_as_str].append(random_employee)
    current_date += datetime.timedelta(days=1)

In [59]:
curr_date = None
for rh in rental_history:
    ret_date = rh["return_date"].split(" ")[0]
    empl = np.random.choice(employees_per_day[ret_date])
    rh["return_employee_id"] = empl.id
    borrow_date = rh["rental_date"].split(" ")[0]
    empl = np.random.choice(employees_per_day[borrow_date])
    rh["employee_id"] = empl.id

In [60]:
orm.Rental.delete().execute()

0

In [61]:
rental_history = sorted(rental_history, key=lambda x: x["rental_date"])
# add to db
orm.Rental.insert_many(rental_history).execute()

## Rental with no return

In [62]:
games_to_rent = orm.GamesToRent.select().where(orm.GamesToRent.rented == True)
len(games_to_rent)

0

In [96]:
# Generate rentals in last week that were not returned

customers = orm.Customers.select()

no_rentals = 24

not_returned = []

for i in range(no_rentals):
    # Get random customer
    customer = np.random.choice(customers)
    # Get random game
    games_to_rent = orm.GamesToRent.select().where(orm.GamesToRent.rented == False)
    game = np.random.choice(games_to_rent)
    # Get random date
    rent_date = shop_end_date - datetime.timedelta(days=np.random.randint(1, 7))
    # Cast rent_date to datetime
    rent_date = datetime.datetime.strptime(rent_date.strftime("%Y-%m-%d") + " " + str(np.random.randint(10, 17)) + ":" + str(np.random.randint(0, 60)) + ":" + str(np.random.randint(0, 60)), "%Y-%m-%d %H:%M:%S")
    # Check is Sunday
    if rent_date.weekday() == 6:
        rent_date -= datetime.timedelta(days=1)

    # game.rented = True
    # game.save()
    not_returned.append({
        "additional_payment_id": None,
        "return_date": None,
        "rental_date": rent_date,
        "customer_id": customer.id,
        "return_employee_id": None,
        "employee_id": np.random.choice(employees_per_day[rent_date.strftime("%Y-%m-%d")]).id,
    })

In [97]:
orm.Rental.insert_many(not_returned).execute()

## Registration

In [64]:
import pandas as pd
top_first_names = pd.read_csv("./data/new-top-firstNames.csv", sep=",")
top_last_names = pd.read_csv("./data/new-top-surnames.csv", sep=",")
top_first_names["newPerct2013"] = top_first_names["newPerct2013"] / top_first_names["newPerct2013"].sum()
top_last_names["newPerct2013"] = top_last_names["perct2013"] / top_last_names["perct2013"].sum()

def get_random_name():
    first_name = np.random.choice(top_first_names["name"], p=top_first_names["newPerct2013"])
    last_name = np.random.choice(top_last_names["name"], p=top_last_names["newPerct2013"])
    # last name is uppercased - fix that
    last_name = last_name[0] + last_name[1:].lower()
    return first_name, last_name
get_random_name()

('Gary', 'Smith')

In [65]:
phone_numbers = set(orm.Customers.select(orm.Customers.phone) + orm.Employees.select(orm.Employees.phone))

In [66]:
no_registrations = 142
registrations = []
for _ in range(no_registrations):
    first_name, last_name = get_random_name()
    phone = "".join([str(np.random.randint(0, 10)) for _ in range(9)])
    if phone[0] == "0":
        phone = str(np.random.randint(1, 10)) + phone[1:]
    while phone in phone_numbers:
        phone = "".join([str(np.random.randint(0, 10)) for _ in range(9)])
        if phone[0] == "0":
            phone = str(np.random.randint(1, 10)) + phone[1:]
    phone_numbers.add(phone)

    email_variant = np.random.choice(["first,last", "first,dot,last", "first,numbers", 
                                      "last,first", "last,numbers", "first,last,numbers", 
                                      "last,first,numbers", "last,dot,first", "first,dot,last,numbers", "last,dot,first,numbers"
                                      ])

    email = ""
    for email_part in email_variant.split(","):
        if email_part == "first":
            email += first_name
        elif email_part == "last":
            email += last_name
        elif email_part == "numbers":
            email += "".join([str(np.random.randint(0, 10)) for _ in range(np.random.randint(1, 4))])
        elif email_part == "dot":
            email += "."

    # Add email provider from gmail outlook etc
    email += "@" + np.random.choice(["gmail", "outlook", "yahoo", "protonmail", "tutanota"]) + ".com"

    registrations.append({
        "first_name": first_name,
        "last_name": last_name,
        "phone": phone,
        "email": email
    })

In [67]:
# Group orm.rental by customer_id
rental_history = orm.Rental.select()
rental_per_customer = {}
for rental in rental_history:
    if rental.customer_id not in rental_per_customer:
        rental_per_customer[rental.customer_id] = []
    rental_per_customer[rental.customer_id].append(rental)

In [68]:
# Get customers with more than 3 rentals
customers_with_more_than_3_rentals = []
for key in rental_per_customer:
    if len(rental_per_customer[key]) >= 3:
        customers_with_more_than_3_rentals.append(key)

In [69]:
len(customers_with_more_than_3_rentals)

317

In [70]:
# Get DB objects of customers with more than 3 rentals
customers_with_more_than_3_rentals = orm.Customers.select().where(orm.Customers.id << customers_with_more_than_3_rentals)


In [71]:
customers_with_more_than_3_rentals[1]

<Customers: 2>

In [72]:
for customer in customers_with_more_than_3_rentals:
    registrations.append({
        "first_name": customer.first_name,
        "last_name": customer.last_name,
        "phone": customer.phone,
        "email": customer.email
    })

In [73]:
orm.Registration.insert_many(registrations).execute()

## Tournaments

In [74]:
tournamets_blueprint = [
    {
        "name": "say UNO!",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("UNO"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "HAPPY LITTLE DINOSAURS survival",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("HAPPY LITTLE DINOSAURS"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "sheREEF of the REEF",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("REEF"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "tonne of CARCASSONNE",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("CARCASSONNE"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "DOOBLE game",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("SPOT IT"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "SET of rivarly",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("SET"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "artful board of AZUL",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("AZUL"))[0].id,
        "entry_fee": 0,
        "player_type": "solo",
        "number_of_participants": None,
        "description": "the most points in 15 games, random opponents"
    },
    {
        "name": "MAGIC MAZE runaways",
        "date": None,
        "game_id": orm.Games.select().where(orm.Games.name.contains("MAGIC MAZE"))[0].id,
        "entry_fee": 0,
        "player_type": "team",
        "number_of_participants": None,
        "description": "the best time in 3 games"
    },
]

In [75]:
shop_start_date = datetime.datetime(2018, 6, 1)
covid_start_date = datetime.datetime(2020, 3, 12)
covid_end_date = datetime.datetime(2020, 6, 1)
covid_start_date2 = datetime.datetime(2020, 11, 1)
covid_end_date2 = datetime.datetime(2021, 3, 1)
shop_end_date = datetime.datetime(2023, 6, 28)

In [76]:
shop_start_date, covid_start_date, covid_end_date, shop_end_date

(datetime.datetime(2018, 6, 1, 0, 0),
 datetime.datetime(2020, 3, 12, 0, 0),
 datetime.datetime(2020, 6, 1, 0, 0),
 datetime.datetime(2023, 6, 28, 0, 0))

In [77]:
# Iterate over days from shop start date to shop end date
current_date = deepcopy(shop_start_date)
days_till_next_tour = np.random.randint(9, 17)
while current_date <= shop_end_date:
    # if date is in covid period continue
    if covid_start_date <= current_date <= covid_end_date:
        current_date += datetime.timedelta(days=1)
        continue
    if covid_start_date2 <= current_date <= covid_end_date2:
        current_date += datetime.timedelta(days=1)
        continue
    if days_till_next_tour == 0:
        # Add new tournament
        tour = np.random.choice(tournamets_blueprint)
        new_tour = orm.Competitions(**{**tour, "date": current_date, "number_of_participants": np.random.randint(20, 50)})
        new_tour.save()
        # Set days till next tournament
        days_till_next_tour = np.random.randint(9, 17) + 1
    days_till_next_tour -= 1
    current_date += datetime.timedelta(days=1)


In [78]:
from peewee import fn

In [79]:
prizes_kind = ['small_board_game', 'medium_board_game', 'big_board_game', 'giftcard', 'logic_puzzle']
participants = []
# Generate prizes for turnaments
tournaments = orm.Competitions.select()
for tour in tournaments:
    # Sample random number of players
    no_players = tour.number_of_participants
    
    # Generate unique random number of points from 0 to 100
    points = np.random.choice(np.arange(0, 200), size=no_players, replace=False)
    # Sort points from biggest to smallest
    points.sort()
    points = points[::-1]

    # Get no_players from registrations
    players = orm.Registration.select().order_by(fn.Random()).limit(no_players)

    for i, player in enumerate(players):
        prize_id = None
        if i < 3:
            # Generate random prize
            prize = orm.Prizes(**{"kind": np.random.choice(prizes_kind), "given": True})
            prize.save()
            prize_id = prize.id

            if np.random.rand() < 0.5:
                # Add extra prize 
                prize = orm.Prizes(**{"kind": np.random.choice(prizes_kind), "given": False}).save()
        present = True

        if np.random.rand() < 0.5 and i >= no_players - 4:
            present = False

        if i > 0 and not participants[-1]["present"]:
            present = False
        participants.append({
            "competition_id": tour.id,
            "registration_id": player.id,
            "score": points[i] if present else 0,
            "ranking": i + 1,
            "prize_id": prize_id,
            "present": present
        })

In [80]:
orm.Participants.insert_many(participants).execute()

## Future competitions

In [81]:
no_comp = 3

prev_diff = 0
for _ in range(no_comp):
    new_tour_date = shop_end_date + datetime.timedelta(days=prev_diff + np.random.randint(9, 17))
    if new_tour_date.weekday() == 6:
        new_tour_date += datetime.timedelta(days=-1)
    blueprint = np.random.choice(tournamets_blueprint)
    new_tour = orm.Competitions(**{
        "name": blueprint["name"],
        "entry_fee": 0.0,
        "number_of_participants": 0,
        "date": new_tour_date,
        "description": blueprint["description"],
        "game_id": blueprint["game_id"],
        }
    )
    new_tour.save()
    prev_diff = (new_tour_date - shop_end_date).days


## Meetings

In [82]:
current_date = deepcopy(shop_start_date)
days_till_next_tour = np.random.randint(9, 17)
end_date = datetime.datetime(2023, 6, 28) + datetime.timedelta(days=90)
meetings = []
while current_date <= shop_end_date:
    if covid_start_date <= current_date <= covid_end_date:
        current_date += datetime.timedelta(days=1)
        continue
    if covid_start_date2 <= current_date <= covid_end_date2:
        current_date += datetime.timedelta(days=1)
        continue
    if days_till_next_tour == 0:
        # Add new tournament
        meetings.append({
            "date": current_date,
            "name": np.random.choice(["News in board games", "Board games creators meeting", "Tournaments discussion session"]),
            "fee": round(np.random.uniform(0, 5), 2),
            "online": np.random.rand() < 0.5,
        })
        # Set days till next tournament
        days_till_next_tour = np.random.randint(9, 17) + 1
    days_till_next_tour -= 1
    current_date += datetime.timedelta(days=1)


In [83]:
orm.Meetings.insert_many(meetings).execute()

In [84]:
for meeting in orm.Meetings.select():
    meeting.date = meeting.date + datetime.timedelta(hours=int(np.random.randint(14, 18)), minutes=int(np.random.choice([0, 30])))
    meeting.save()